In [2]:
import numpy as np

In [38]:
from pyhive import hive 
host_name = "192.168.1.10"
port = 10000
user = "cloudera"
password = "cloudera"
database="default"

In [39]:
 conn = hive.Connection(host=host_name, port=port, username=user, 
                        password=password,database=database, auth='CUSTOM')

In [40]:
import pandas as pd

* Access to hive table 

In [45]:
def select(table):
    cur = conn.cursor()
    cur.execute('desc {}'.format(table))
    result = cur.fetchall()
    cols = [r[0] for r in result]
    cur.execute('select * from  {}'.format(table))
    result = cur.fetchall()
    df = pd.DataFrame(result, columns=cols)
    return df  

In [32]:
houses = select('houses')
houses.head()

,city,area,rooms,bathroom,parking_spaces,floor,animal,furniture,hoa,rent_amount,property_tax,fire_insurance,total
0,1,240,3,3,4,-,acept,furnished,0.0,8000.0,1000.0,121.0,9121.0
1,0,64,2,1,1,10,acept,not furnished,540.0,820.0,122.0,11.0,1493.0
2,1,443,5,5,4,3,acept,furnished,4172.0,7000.0,1417.0,89.0,12680.0
3,1,73,2,2,1,12,acept,not furnished,700.0,1250.0,150.0,16.0,2116.0
4,1,19,1,1,0,-,not acept,not furnished,0.0,1200.0,41.0,16.0,1257.0


In [33]:
houses.describe()

,city,area,rooms,bathroom,parking_spaces,hoa,rent_amount,property_tax,fire_insurance,total
count,6080.000000,6080.000000,6080.000000,6080.000000,6080.000000,6080.000000,6080.000000,6080.000000,6080.000000,6080.000000
mean,0.863322,151.143914,2.492599,2.341612,1.756250,1088.426480,4395.844408,490.333388,58.204276,6033.194572
std,0.343535,375.559485,1.129665,1.438860,1.611909,3981.357627,3576.668946,6212.564538,49.313441,8751.347700
min,0.000000,10.000000,1.000000,1.000000,0.000000,0.000000,420.000000,0.000000,3.000000,660.000000
25%,1.000000,58.000000,2.000000,1.000000,1.000000,24.500000,1800.000000,33.000000,23.000000,2451.500000
50%,1.000000,100.000000,3.000000,2.000000,1.000000,650.000000,3111.000000,145.000000,41.000000,4128.500000
75%,1.000000,200.000000,3.000000,3.000000,2.000000,1436.000000,5952.500000,458.000000,77.000000,8074.000000
max,1.000000,24606.000000,10.000000,10.000000,12.000000,220000.000000,45000.000000,366300.000000,677.000000,372700.000000


* percentile calculation

In [41]:
var = 'total'
pref = 'P_'
pctl_from, pctl_to, pctl_by = 0, 100, 5 
pctl_range = np.arange(pctl_from, pctl_to + pctl_by, pctl_by)
pctl_range = pctl_range[pctl_range<=100]
pctl_list = tuple(r/100 for r in pctl_range)
hv_arr = 'array'+str(pctl_list)
hv_arr

'array(0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0)'

In [42]:
cur = conn.cursor()

In [43]:
cols = [pref+str(r) for r in pctl_range]

In [44]:
cur.execute("drop table quantiles")
qr = """ create temporary table quantiles as 
    select percentile(cast( {var} as bigint), {hv_arr}) as pctls from {table}
""".format(var=var,hv_arr=hv_arr,table='houses')
cur.execute(qr)
cur.execute("select pctls from quantiles")
result = cur.fetchall()
result


[('[660.0,1251.0,1573.0,1866.8500000000001,2165.0,2451.5,2692.7,3027.3,3399.0,3788.55,4128.5,4618.900000000001,5209.4,5985.75,6871.6,8074.0,9238.2,10731.499999999996,12860.0,15931.500000000005,372700.0]',)]

In [46]:
select('quantiles')

,pctls
0,"[660.0,1251.0,1573.0,1866.8500000000001,2165.0..."


In [47]:
arr = ', '.join( ['pctls[{}] as {}{}'.format(i,pref,r) for i,r in enumerate(pctl_range)])

In [48]:
arr

'pctls[0] as P_0, pctls[1] as P_5, pctls[2] as P_10, pctls[3] as P_15, pctls[4] as P_20, pctls[5] as P_25, pctls[6] as P_30, pctls[7] as P_35, pctls[8] as P_40, pctls[9] as P_45, pctls[10] as P_50, pctls[11] as P_55, pctls[12] as P_60, pctls[13] as P_65, pctls[14] as P_70, pctls[15] as P_75, pctls[16] as P_80, pctls[17] as P_85, pctls[18] as P_90, pctls[19] as P_95, pctls[20] as P_100'

In [49]:
qr = """ create  table percentiles_calc as 
        select {} from quantiles
     """.format(arr)
qr


' create  table percentiles_calc as \n        select pctls[0] as P_0, pctls[1] as P_5, pctls[2] as P_10, pctls[3] as P_15, pctls[4] as P_20, pctls[5] as P_25, pctls[6] as P_30, pctls[7] as P_35, pctls[8] as P_40, pctls[9] as P_45, pctls[10] as P_50, pctls[11] as P_55, pctls[12] as P_60, pctls[13] as P_65, pctls[14] as P_70, pctls[15] as P_75, pctls[16] as P_80, pctls[17] as P_85, pctls[18] as P_90, pctls[19] as P_95, pctls[20] as P_100 from quantiles\n     '

In [50]:
cur.execute(qr)
# result = cur.fetchall()
# result

In [51]:
select('percentiles_calc')

,p_0,p_5,p_10,p_15,p_20,p_25,p_30,p_35,p_40,p_45,...,p_55,p_60,p_65,p_70,p_75,p_80,p_85,p_90,p_95,p_100
0,660.0,1251.0,1573.0,1866.85,2165.0,2451.5,2692.7,3027.3,3399.0,3788.55,...,4618.9,5209.4,5985.75,6871.6,8074.0,9238.2,10731.5,12860.0,15931.5,372700.0


* checking percentile with numpy 

In [52]:
houses = select('houses')
houses.head()

,city,area,rooms,bathroom,parking_spaces,floor,animal,furniture,hoa,rent_amount,property_tax,fire_insurance,total
0,1,240,3,3,4,-,acept,furnished,0.0,8000.0,1000.0,121.0,9121.0
1,0,64,2,1,1,10,acept,not furnished,540.0,820.0,122.0,11.0,1493.0
2,1,443,5,5,4,3,acept,furnished,4172.0,7000.0,1417.0,89.0,12680.0
3,1,73,2,2,1,12,acept,not furnished,700.0,1250.0,150.0,16.0,2116.0
4,1,19,1,1,0,-,not acept,not furnished,0.0,1200.0,41.0,16.0,1257.0


In [53]:
np.percentile(houses['total'], pctl_range )

array([   660.  ,   1251.  ,   1573.  ,   1866.85,   2165.  ,   2451.5 ,
         2692.7 ,   3027.3 ,   3399.  ,   3788.55,   4128.5 ,   4618.9 ,
         5209.4 ,   5985.75,   6871.6 ,   8074.  ,   9238.2 ,  10731.5 ,
        12860.  ,  15931.5 , 372700.  ])